In [20]:
import evaluate
import process
text = 'data/brown.txt'

In [21]:
import nltk
from nltk.corpus import brown
from nltk.tokenize import casual

import json

from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import numpy as np
import argparse
import numpy as np
from scipy.stats import spearmanr

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import cosine
from numpy.linalg import norm

import random
import os

from process import load_model, load_msr

In [22]:
class MySentences(object):
    def __init__(self, filename):
        self.filename = filename
 
    def __iter__(self):
        for line in open(self.filename, 'r'):
            yield [word.lower() for word in line.split()]
sentence = MySentences(text)

In [13]:
windowList = [2,5,10]
dimList = [100,300,1000]
EPOCHS = 5
iwin = 2
idim = 100

In [30]:
# count unigrams
from collections import Counter
word_counts = Counter()
for x in sentence:
    word_counts.update(x)

In [25]:
# count co-occurance

In [31]:
len(word_counts)

49815

In [32]:
word_counts["that"]

10594

In [7]:
text = "sentences iterable can be simply a list, but for larger corpora, consider a generator that streams the sentences directly from disk/network"